In [1]:
import mxnet as mx
import boto3

# define training batch size
batch_size = 100

# Using AWS for CRISP-DM Phases 3-5: Data Prepartion, Modeling, and Evaluation

Now that you are inside a Jupyter Notebook, we assume that most of you should be back within familiar territory. As such, this tutorial will not go into detail about these phases. Rather, we'll quickly breeze through these three phases with a focus on how to get your model prepared for Phase 6, Deployment. In that phase, we'll provide more detail on how to deploy real-time models on the AWS architecture.

Because this tutorial is focused on the cloud computing architecture rather than the Data Science, we'll use a common dataset, MNIST, and train an image classifier using MXNet.

## 1. Standard CRISP-DM Phases 3-5

As mentioned above, this tutorial will not focus on the Data Science of the modeling. As such, the following section training and evaluation code is 90% taken from https://mxnet.incubator.apache.org/tutorials/python/mnist.html

If you are familiar with MXNet and the standard training and evaluation code, feel free to jump ahead.

### Phase 3: Data Preparation

#### Load Data

In [2]:
mnist = mx.test_utils.get_mnist()

#### Split training/test sets

In [3]:
ntrain = int(mnist['train_data'].shape[0]*0.8)
X_train = mnist['train_data'][:ntrain]
y_train = mnist['train_label'][:ntrain]
X_test = mnist['train_data'][ntrain:]
y_test = mnist['train_label'][ntrain:]

### Phase 4: Model Training

#### Define Network

In [4]:
data = mx.sym.var('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(5,5), num_filter=20)
tanh1 = mx.sym.Activation(data=conv1, act_type="tanh")
pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=50)
tanh2 = mx.sym.Activation(data=conv2, act_type="tanh")
pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
tanh3 = mx.sym.Activation(data=fc1, act_type="tanh")
# second fullc
fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=10)
# softmax loss
lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

#### Train Network

In [5]:
# create iterator around training and validation data
train_iter = mx.io.NDArrayIter(mnist['train_data'][:ntrain], mnist['train_label'][:ntrain], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['train_data'][ntrain:], mnist['train_label'][ntrain:], batch_size)

# create a trainable module on GPU 0
lenet_model = mx.mod.Module(symbol=lenet, context=mx.gpu())
# train with the same
lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.1},
                eval_metric='acc',
                batch_end_callback = mx.callback.Speedometer(batch_size, 100),
                num_epoch=10)

### Phase 5: Model evaluation

In [6]:
# predict accuracy for lenet
acc = mx.metric.Accuracy()
lenet_model.score(val_iter, acc)
print(acc)

EvalMetric: {'accuracy': 0.9854166666666667}


## 2. Store Trained Model

If this model looks good then let's store the model.

In [7]:
lenet.save('./mnist_symbol.mxnet')
lenet_model.save_params('./mnist_module.mxnet')

s3 = boto3.client('s3')
s3.upload_file(
    './mnist_symbol.mxnet',
    'jakechenawstemp',
    'mnist_symbol.mxnet'
)
s3.upload_file(
    './mnist_module.mxnet',
    'jakechenawstemp',
    'mnist_module.mxnet'
)

Continue to Deployment on Lambda